In [1]:
import os

In [4]:
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/pyCharmProjects/Chicken-Disease-Classification--Project'

In [3]:
os.chdir('../')

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_path: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.utils import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_path="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [10]:
from urllib.parse import urlparse

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_path,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path):
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy': self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [16]:
try:
    config = ConfigurationManager()
    val_config =config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2023-12-18 13:38:23,444: INFO: utils: yaml file: config/config.yaml loaded successfully]
[2023-12-18 13:38:23,448: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-12-18 13:38:23,450: INFO: utils: created directory at: artifacts]
Found 78 images belonging to 2 classes.
5/5 [==============================] - 0s 90ms/step - loss: 1.2700 - accuracy: 0.5000
[2023-12-18 13:38:24,295: INFO: utils: json file saved at: scores.json]
